<div class="alert alert-block alert-success">
<b>PREP: </b>Scenario analysis
</div>

In [1]:
# import json
# import numpy as np
# import os
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# sns.set_theme(style="darkgrid")


# def time_to_minutes(timestr):
#     # Handle '+1' suffix by removing it before parsing
#     timestr = timestr.split('+')[0]  # Remove '+1' if present
#     hh, mm = timestr.split(':')
#     return int(hh) * 60 + int(mm)

# def calculate_slack_for_scenario(scenario_data):
#     """
#     Calculate the slack metric for the given scenario.
    
#     Slack is defined as:
#         Slack = 1 - (total flight minutes in recovery period / total recovery period aircraft-minutes)
    
#     A slack of 1 means no flights during recovery period.
#     A slack of 0 means flights occupy the entire recovery period.
#     """

#     def time_to_minutes(timestr):
#         # Handle '+1' suffix by removing it before parsing
#         timestr = timestr.split('+')[0]  # Remove '+1' if present
#         hh, mm = timestr.split(':')
#         return int(hh) * 60 + int(mm)
#     # Extract scenario start/end times
#     # We assume the same date for start and end for simplicity.
#     recovery_start_time_str = scenario_data["recovery_start_time"]  
#     recovery_end_time_str = scenario_data["recovery_end_time"]      
    
#     recovery_start_minutes = time_to_minutes(recovery_start_time_str)
#     recovery_end_minutes = time_to_minutes(recovery_end_time_str)
#     total_recovery_period_minutes = recovery_end_minutes - recovery_start_minutes
    
#     total_aircraft = scenario_data["total_aircraft"]
    
#     # Calculate total flight minutes within the recovery period
#     flights = scenario_data["flights"]
#     total_flights = len(flights)
#     total_flight_minutes_in_recovery = 0
#     total_flight_minutes_total = 0
    
#     for flight_id, flight_data in flights.items():
#         dep_time_str = flight_data["DepTime"]  
#         arr_time_str = flight_data["ArrTime"] 
        
#         dep_minutes = time_to_minutes(dep_time_str)
#         arr_minutes = time_to_minutes(arr_time_str)
        
#         total_flight_minutes_total += arr_minutes - dep_minutes
#         overlap_start = max(dep_minutes, recovery_start_minutes)
#         overlap_end = min(arr_minutes, recovery_end_minutes)
        
#         if overlap_end > overlap_start:
#             flight_overlap = overlap_end - overlap_start
#         else:
#             flight_overlap = 0
        
#         total_flight_minutes_in_recovery += flight_overlap
    
#     # Calculate total aircraft-minutes available during the recovery period
#     total_recovery_aircraft_minutes = total_recovery_period_minutes * total_aircraft
    
#     # Slack calculation
#     if total_recovery_aircraft_minutes == 0:
#         slack = 1.0
#     else:
#         slack = 1 - (total_flight_minutes_in_recovery / total_recovery_aircraft_minutes)
    
#     return slack, total_flights, total_flight_minutes_total


# def extract_disruption_stats(scenario_data):
#     """
#     Extract disruption statistics:
#     - Count of fully disrupted (prob = 1.0)
#     - Count of uncertain disruptions (0 < prob < 1.0)
#     - Average probability across all aircraft (where an aircraft's probability is the max disruption probability it faces, 
#       with 1.0 for fully disrupted and 0.0 if no disruption)
#     - Average uncertainty probability (average of all disruptions where 0<prob<1.0, excluding 0 and 1)
#     """
#     disruptions_info = scenario_data.get('disruptions', {})
#     disruptions_list = disruptions_info.get('disruptions', [])
#     total_aircraft = disruptions_info.get('total_aircraft', 0)

#     if total_aircraft == 0:
#         # No aircraft or no disruptions
#         return 0, 0, 0.0, 0.0

#     fully_disrupted_count = sum(1 for d in disruptions_list if d.get('probability', 0.0) == 1.0)
#     uncertain_disruptions = [d for d in disruptions_list if 0.0 < d.get('probability', 0.0) < 1.0]
#     uncertain_count = len(uncertain_disruptions)

#     aircraft_ids = scenario_data.get('aircraft_ids', [])
#     ac_prob_map = {ac: 0.0 for ac in aircraft_ids}  
    
#     for d in disruptions_list:
#         ac_id = d.get('aircraft_id')
#         p = d.get('probability', 0.0)
#         # Keep the max probability for that aircraft
#         if ac_id in ac_prob_map:
#             ac_prob_map[ac_id] = max(ac_prob_map[ac_id], p)

#     avg_ac_prob = sum(ac_prob_map.values()) / total_aircraft if total_aircraft > 0 else 0.0

#     # Average uncertainty probability (only consider disruptions where 0<prob<1)
#     if len(uncertain_disruptions) > 0:
#         avg_uncertainty_prob = np.mean([d['probability'] for d in uncertain_disruptions])
#     else:
#         avg_uncertainty_prob = 0.0

#     return fully_disrupted_count, uncertain_count, avg_ac_prob, avg_uncertainty_prob, total_aircraft

# # Path to the scenarios folder
# scenario_folder_path = "../logs/scenarios/"
# latest_folder = max(
#     [f for f in os.listdir(scenario_folder_path) if f.startswith("scenario_folder_")],
#     key=lambda x: int(x.split('_')[-1].replace('.json', ''))
# )

# latest_folder = "scenario_folder_scenario_84.json"

# file_path = os.path.join(scenario_folder_path, latest_folder)

# # Extract scenario ID
# scenario_id = file_path.split('_')[-1].split('.')[0]
# print(f"Scenario ID: {scenario_id}")

# # Load the JSON data
# with open(file_path, 'r') as file:
#     data = json.load(file)

# # Extract the scenarios from the JSON data
# scenarios = data['outputs']


# # Extract the data_folder (not strictly necessary for slack calculation, but we print it for context)
# data_folder = data['data_folder']
# print(f"Data Folder: {data_folder}")

# # Calculate slack and disruption stats for each scenario and store in a list of dicts
# results = []
# for scenario_name, scenario_data in scenarios.items():
#     scenario_slack, total_flights, total_flight_minutes_total = calculate_slack_for_scenario(scenario_data)
#     fully_disrupted_count, uncertain_count, avg_ac_prob, avg_uncertain_prob, total_aircraft = extract_disruption_stats(scenario_data)
#     results.append({
#         "Scenario": scenario_name,
#         "ScenarioSlack": scenario_slack,
#         "TotalFlights": total_flights,
#         "TotalFlightMinutes": total_flight_minutes_total,
#         "FullyDisruptedCount": fully_disrupted_count,
#         "UncertainCount": uncertain_count,
#         "AvgAircraftProbability": avg_ac_prob,
#         "AvgUncertaintyProbability": avg_uncertain_prob,
#         "TotalAircraft": total_aircraft
#     })

# # Convert results to DataFrame
# scenarios_df = pd.DataFrame(results)
# print(scenarios_df)

# # Save the slack results to CSV
# # output_file = os.path.join(scenario_folder_path, f"scenario_slack_metrics_{scenario_id}.csv")
# # scenarios_df.to_csv(output_file, index=False)
# # print(f"Slack metrics saved to {output_file}")


<div class="alert alert-block alert-success">
<b>RUN: </b>Inferencing
</div>

In [ ]:
import os
import json
import time
import numpy as np
import pandas as pd
import torch
from datetime import datetime
import time
import json
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="darkgrid")
import os
import os
import sys
sys.path.append('..')

import json
import time
import numpy as np
import pandas as pd
import torch
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="darkgrid")

from stable_baselines3 import DQN
from src.environment import AircraftDisruptionEnv
from stable_baselines3 import DQN
from src.environment import AircraftDisruptionEnv
from scripts.utils import load_scenario_data, NumpyEncoder, get_training_metadata
from scripts.logger import create_new_id, log_inference_metadata, find_corresponding_training_id, convert_to_serializable

def run_inference_dqn_single(model_path, scenario_folder, env_type, seed):
    """
    Runs inference on a single scenario and returns the total reward.
    """
    start_time = time.time()
    data_dict = load_scenario_data(scenario_folder)
    aircraft_dict = data_dict['aircraft']
    flights_dict = data_dict['flights']
    rotations_dict = data_dict['rotations']
    alt_aircraft_dict = data_dict['alt_aircraft']
    config_dict = data_dict['config']

    # print(f"*** initializing env with env_type: {env_type}")

    env = AircraftDisruptionEnv(
        aircraft_dict, flights_dict, rotations_dict, alt_aircraft_dict, config_dict, env_type=env_type
    )

    model = DQN.load(model_path)
    model.set_env(env)
    model.policy.set_training_mode(False)
    model.exploration_rate = 0.0

    # Set seeds for reproducibility
    np.random.seed(seed)
    torch.manual_seed(seed)

    obs, _ = env.reset()
    done_flag = False
    total_reward = 0
    step_num = 0
    max_steps = 1000

    while not done_flag and step_num < max_steps:
        action_mask = obs['action_mask']
        obs = {key: np.array(value, dtype=np.float32) for key, value in obs.items()}
        obs_tensor = model.policy.obs_to_tensor(obs)[0]
        q_values = model.policy.q_net(obs_tensor).detach().cpu().numpy().squeeze()

        masked_q_values = q_values.copy()
        masked_q_values[action_mask == 0] = -np.inf

        # If no valid actions remain, break out
        if np.all(np.isinf(masked_q_values)):
            break
        valid_actions = np.where(masked_q_values != -np.inf)[0]
        action = np.random.choice(valid_actions)
        obs, reward, terminated, truncated, info = env.step(action)
        total_reward += reward
        done_flag = terminated or truncated
        step_num += 1

    total_delays = env.scenario_wide_delay_minutes
    total_cancelled_flights = env.scenario_wide_cancelled_flights
    end_time = time.time()
    scenario_time = end_time - start_time
    scenario_steps = env.scenario_wide_steps
    scenario_resolved_conflicts = env.scenario_wide_resolved_conflicts
    solution_slack = env.scenario_wide_solution_slack
    scenario_wide_tail_swaps = env.scenario_wide_tail_swaps
    scenario_wide_actual_disrupted_flights = env.scenario_wide_actual_disrupted_flights
    scenario_wide_reward_components = env.scenario_wide_reward_components

    return total_reward, total_delays, total_cancelled_flights, scenario_time, scenario_steps, scenario_resolved_conflicts, solution_slack, scenario_wide_tail_swaps, scenario_wide_actual_disrupted_flights, scenario_wide_reward_components


import os
from scripts.utils import load_scenario_data
from src.environment import AircraftDisruptionOptimizer
from scripts.visualizations import StatePlotter
from datetime import datetime
import time

def run_exact_single(scenario_folder):
    start_time = time.time()
    
    # Load scenario data first
    data_dict = load_scenario_data(scenario_folder)
    aircraft_dict = data_dict['aircraft']
    flights_dict = data_dict['flights']
    rotations_dict = data_dict['rotations']
    alt_aircraft_dict = data_dict['alt_aircraft']
    config_dict = data_dict['config']
    
    # Parse recovery period times
    recovery_period = config_dict['RecoveryPeriod']
    start_datetime = datetime.strptime(f"{recovery_period['StartDate']} {recovery_period['StartTime']}", '%d/%m/%y %H:%M')
    end_datetime = datetime.strptime(f"{recovery_period['EndDate']} {recovery_period['EndTime']}", '%d/%m/%y %H:%M')
    
    # Initialize optimizer with loaded data
    optimizer = AircraftDisruptionOptimizer(
        aircraft_dict=aircraft_dict,
        flights_dict=flights_dict,
        rotations_dict=rotations_dict,
        alt_aircraft_dict=alt_aircraft_dict,
        config_dict=config_dict
    )
    
    solution = optimizer.solve()
    
    # Convert optimizer solution to state plotter format
    swapped_flights = [(flight_id, new_aircraft) for flight_id, new_aircraft in solution['assignments'].items()]
    environment_delayed_flights = set(solution['delays'].keys())
    cancelled_flights = set(solution['cancellations'])
    
    # Update flights dict with delays
    updated_flights_dict = flights_dict.copy()
    for flight_id, delay in solution['delays'].items():
        if flight_id in updated_flights_dict:
            flight_info = updated_flights_dict[flight_id]
            flight_info['Delay'] = delay

    total_reward = optimizer.scenario_wide_reward_total
    total_delays = optimizer.scenario_wide_delay_minutes
    total_cancelled_flights = optimizer.scenario_wide_cancelled_flights
    scenario_steps = optimizer.scenario_wide_steps
    scenario_resolved_conflicts = optimizer.scenario_wide_resolved_conflicts
    solution_slack = optimizer.scenario_wide_solution_slack
    scenario_wide_tail_swaps = optimizer.scenario_wide_tail_swaps
    scenario_wide_actual_disrupted_flights = optimizer.scenario_wide_actual_disrupted_flights
    scenario_wide_reward_components = optimizer.scenario_wide_reward_components

    end_time = time.time()
    scenario_time = end_time - start_time

    return total_reward, total_delays, total_cancelled_flights, scenario_time, scenario_steps, scenario_resolved_conflicts, solution_slack, scenario_wide_tail_swaps, scenario_wide_actual_disrupted_flights, scenario_wide_reward_components


def run_inference_on_data_folder(model_paths, data_folder, seeds):
    """
    Runs inference on all scenarios found in 'data_folder', for each model in 'model_paths' and each seed in 'seeds'.

    Args:
        model_paths (list): List of tuples containing (model_path, env_type).
        data_folder (str): Path to the folder containing scenario subfolders.
        seeds (list): List of seeds for reproducibility.

    Returns:
        pd.DataFrame: A DataFrame containing scenario, model, seed, and total reward.
    """

    # Identify all scenario folders within data_folder
    scenario_folders = [
        os.path.join(data_folder, folder)
        for folder in os.listdir(data_folder)
        if os.path.isdir(os.path.join(data_folder, folder))
    ]

    num_combinations = len(model_paths) * len(seeds) * len(scenario_folders)
    print(f"Number of combinations: {num_combinations}")
    current_combination = 0

    results = []
    for scenario_folder in scenario_folders:
        scenario_name = os.path.basename(scenario_folder)
        for model_tuple in model_paths:
            model_path, env_type = model_tuple  # Unpack the tuple correctly
            for seed in seeds:
                current_combination += 1
                print(f"({current_combination}/{num_combinations} - {round(current_combination/num_combinations*100, 2)}%)")
                print(f"env_type: {env_type}, model_path: {model_path}, scenario_folder: {scenario_folder}, seed: {seed}")
                if env_type == "exact":
                    total_reward, total_delays, total_cancelled_flights, scenario_time, scenario_steps, scenario_resolved_conflicts, solution_slack, scenario_wide_tail_swaps, scenario_wide_actual_disrupted_flights, scenario_wide_reward_components = run_exact_single(scenario_folder)
                else:
                    total_reward, total_delays, total_cancelled_flights, scenario_time, scenario_steps, scenario_resolved_conflicts, solution_slack, scenario_wide_tail_swaps, scenario_wide_actual_disrupted_flights, scenario_wide_reward_components = run_inference_dqn_single(model_path, scenario_folder, env_type, seed)
                results.append({
                    "Scenario": scenario_name,
                    "Model": os.path.basename(model_path),
                    "Seed": seed,
                    "TotalReward": total_reward,
                    "TotalDelays": total_delays,
                    "TotalCancelledFlights": total_cancelled_flights,
                    "ScenarioTime": scenario_time,
                    "ScenarioSteps": scenario_steps,
                    "ScenarioResolvedConflicts": scenario_resolved_conflicts,
                    "SolutionSlack": solution_slack,
                    "TailSwaps": scenario_wide_tail_swaps,
                    "ActualDisruptedFlights": scenario_wide_actual_disrupted_flights,
                    "Reward_delay_penalty_total": scenario_wide_reward_components["delay_penalty_total"],
                    "Reward_cancel_penalty": scenario_wide_reward_components["cancel_penalty"],
                    "Reward_inaction_penalty": scenario_wide_reward_components["inaction_penalty"],
                    "Reward_proactive_bonus": scenario_wide_reward_components["proactive_bonus"],
                    "Reward_time_penalty": scenario_wide_reward_components["time_penalty"],
                    "Reward_final_conflict_resolution_reward": scenario_wide_reward_components["final_conflict_resolution_reward"]
                })

    results_df = pd.DataFrame(results)
    return results_df



ac = "20"

# Define models and seeds 
model_paths = [
    # ("../22-run/6ac-10000-deterministic/myopic_2023.zip", "myopic"),
    # ("../22-run/6ac-10000-deterministic/proactive_2023.zip", "proactive"),
    # ("../22-run/6ac-10000-deterministic/reactive_2023.zip", "reactive"),
    # ("../trained_models/dqn/6ac-700-diverse/1023/myopic-89.zip", "myopic"),
    # ("../trained_models/dqn/6ac-700-diverse/1023/proactive-93.zip", "proactive"), 
    # # ("../trained_models/dqn/6ac-700-diverse/1023/reactive-96.zip", "reactive"),
    # ("../trained_models/dqn/6ac-700-diverse/1024/myopic-90.zip", "myopic"),
    # # ("../trained_models/dqn/6ac-700-diverse/1024/proactive-92.zip", "proactive"), 
    # ("../trained_models/dqn/6ac-700-diverse/1024/reactive-95.zip", "reactive"),
    # ("../trained_models/dqn/6ac-700-diverse/1025/myopic-91.zip", "myopic"),
    #     # ("../trained_models/dqn/6ac-700-diverse/1023/myopic-89.zip", "myopic"),
    # ("../trained_models/dqn/6ac-700-diverse/1023/proactive-93.zip", "proactive"), 
    # ("../trained_models/dqn/6ac-700-diverse/1023/reactive-96.zip", "reactive"),
    # ("../trained_models/dqn/6ac-700-diverse/1024/myopic-90.zip", "myopic"),
    # ("../trained_models/dqn/6ac-700-diverse/1024/proactive-92.zip", "proactive"), 
    # ("../trained_models/dqn/6ac-700-diverse/1024/reactive-95.zip", "reactive"), 
    (f"../1-bbbb-multi-ac-6ac-run/{ac}ac-10-multi-ac-deterministic/myopic_2023.zip", "myopic"),
    (f"../1-bbbb-multi-ac-6ac-run/{ac}ac-10-multi-ac-deterministic/proactive_2023.zip", "proactive"),
    (f"../1-bbbb-multi-ac-6ac-run/{ac}ac-10-multi-ac-deterministic/reactive_2023.zip", "reactive"),
    ("exact", "exact")
]

# seeds = [25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]
seeds = []
for i in range(1, 2):
    seeds.append(i)

data_folder = f"../data/TEMPOOOOOO/{ac}ac-10-multi-ac-deterministic/"

print(seeds)
# approx 2,5min per seed with 4 models

results_df = run_inference_on_data_folder(model_paths, data_folder, seeds)


<div class="alert alert-block alert-success">
</br>
</br>
</br>
<b>DONE: </b>MERGED DATASET
</br>
</br>
</br>
</br>
</div>

In [ ]:
import pandas as pd
print(results_df.columns)
# Assuming results_df is already defined and contains the necessary data
# Create a new DataFrame to store the runtimes and runtime per step
runtimes_df = pd.DataFrame(columns=['ac', 'model', 'runtime', 'runtime_per_step'])



# Get unique models from results_df
unique_models = results_df['Model'].unique()

# For each unique model, add two rows to runtimes_df with the runtime and runtime per step
for model in unique_models:
    # Assuming 'ScenarioTime' represents the runtime and 'ScenarioSteps' represents the number of steps
    runtime = results_df[results_df['Model'] == model]['ScenarioTime'].mean()
    scenario_steps = results_df[results_df['Model'] == model]['ScenarioSteps'].mean()
    runtime_per_step = runtime / scenario_steps if scenario_steps > 0 else 0  # Avoid division by zero
    new_row = {'ac': ac, 'model': model, 'runtime': runtime, 'runtime_per_step': runtime_per_step}
    runtimes_df = pd.concat([runtimes_df, pd.DataFrame([new_row])], ignore_index=True)
import os

# Check if the file exists
if not os.path.exists('runtimes_abc.csv'):
    # If not, create a new file
    runtimes_df.to_csv('runtimes_abc.csv', index=False)
else:
    # If the file exists, append new rows
    runtimes_df.to_csv('runtimes_abc.csv', mode='a', index=False, header=False)
